The following code loads the olympics dataset (olympics.csv), which was derrived from the Wikipedia entry on [All Time Olympic Games Medals](https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table), and does some basic data cleaning. 

The columns are organized as # of Summer games, Summer medals, # of Winter games, Winter medals, total # number of games, total # of medals.

In [10]:
import pandas as pd

df = pd.read_csv('olympics.csv', index_col=0, skiprows=1)

for col in df.columns:
    if col[:2]=='01':
        df.rename(columns={col:'Gold'+col[4:]}, inplace=True)
    if col[:2]=='02':
        df.rename(columns={col:'Silver'+col[4:]}, inplace=True)
    if col[:2]=='03':
        df.rename(columns={col:'Bronze'+col[4:]}, inplace=True)
    if col[:1]=='№':
        df.rename(columns={col:'#'+col[1:]}, inplace=True)

names_ids = df.index.str.split('\s\(') # split the index by '('

df.index = names_ids.str[0] # the [0] element is the country name (new index) 
df['ID'] = names_ids.str[1].str[:3] # the [1] element is the abbreviation or ID (take first 3 characters from that)

df = df.drop('Totals')
df.head()

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
Afghanistan,13,0,0,2,2,0,0,0,0,0,13,0,0,2,2,AFG
Algeria,12,5,2,8,15,3,0,0,0,0,15,5,2,8,15,ALG
Argentina,23,18,24,28,70,18,0,0,0,0,41,18,24,28,70,ARG
Armenia,5,1,2,9,12,6,0,0,0,0,11,1,2,9,12,ARM
Australasia,2,3,4,5,12,0,0,0,0,0,2,3,4,5,12,ANZ


### Part 1
Which country has won the most gold medals in summer games?

In [4]:
def part_one():
    sorted_df = df.sort_values(by='Gold', ascending=False)
    return sorted_df.index[0]
part_one()

'United States'

### Part 2
Which country had the biggest difference between their summer and winter gold medal counts?

In [15]:
def part_two():
    df['difference_summer_winter_gold']=[df['Gold'][n]-df['Gold.1'][n] for n in df.index]
    sorted_df = df.sort_values(by='difference_summer_winter_gold', ascending=False)
    print("Largest difference is: " + str(sorted_df['difference_summer_winter_gold'][0]))
    return sorted_df.index[0]
part_two()

Largest difference is: 880


'United States'

### Part 3
Which country has the biggest difference between their summer gold medal counts and winter gold medal counts relative to their total gold medal count? 

$$\frac{Summer~Gold - Winter~Gold}{Total~Gold}$$

Only considering countries that have won at least 1 gold in both summer and winter.

In [17]:
def part_three():
    df_consider=df[(df['Gold.1'] >=1) & (df['Gold'] >=1)]
    df_consider['difference_summer_winter_gold']=[(df_consider['Gold'][n]-df_consider['Gold.1'][n])/df_consider['Gold'][n] 
                                                  for n in df_consider.index]
    sorted_df = df_consider.sort_values(by='difference_summer_winter_gold', ascending=False)
    return sorted_df.index[0]
part_three()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


'Bulgaria'

### Part 4
Created a Series called "Points" which is a weighted value where each gold medal (`Gold.2`) counts for 3 points, silver medals (`Silver.2`) for 2 points, and bronze medals (`Bronze.2`) for 1 point. The function returns only the column (a Series object) which is newly created, with the country names as indices.

In [18]:
def part_four():
    df['points']=[3*(df['Gold.2'][n])+(2*df['Silver.2'][n])+df['Bronze.2'][n] for n in df.index]
    sorted_df = df.sort_values(by='points', ascending=False)
    return sorted_df['points']
part_four()

United States    5684
Soviet Union     2526
Great Britain    1574
Germany          1546
France           1500
                 ... 
Bahrain             1
Barbados            1
Niger               1
Macedonia           1
Eritrea             1
Name: points, Length: 146, dtype: int64

For the next half of the workbook, I will be using census data from the [United States Census Bureau](http://www.census.gov). Counties are political and geographic subdivisions of states in the United States. This dataset contains population data for counties and states in the US from 2010 to 2015. [See this document](https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2010-2015/co-est2015-alldata.pdf) for a description of the variable names.

In [19]:
census_df = pd.read_csv('census.csv')
census_df.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
0,40,3,6,1,0,Alabama,Alabama,4779736,4780127,4785161,...,0.002295,-0.193196,0.381066,0.582002,-0.467369,1.030015,0.826644,1.383282,1.724718,0.712594
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,...,7.242091,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,...,14.832960,17.647293,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,...,-4.728132,-2.500690,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,...,-5.527043,-5.068871,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861


### Part 5
Which state has the most counties in it?

In [38]:
def part_five():
    df = census_df[census_df.COUNTY!=0].groupby('STNAME').agg({'COUNTY': 'nunique'})
    sorted_df = df.sort_values(by='COUNTY', ascending=False)
    print("Highest number of counties " + str(sorted_df.COUNTY[0]))
    return sorted_df.index[0]
part_five()

Highest number of counties 254


'Texas'

### Part 6
What are the three most populous states (in order of highest population to lowest population)? 

In [39]:
def part_six():
    counties_pop = census_df[census_df['SUMLEV'] == 50].groupby(['STNAME']).sum()['CENSUS2010POP']
    sorted_counties_by_pop = counties_pop.to_frame().sort_values(by='CENSUS2010POP', ascending=False)
    return list(sorted_counties_by_pop.index[0:3])
part_six()

['California', 'Texas', 'New York']

### Part 7
Which county has had the largest absolute change in population within the period 2010-2015?

Note: If County Population in the 5 year period is 100, 120, 80, 105, 100, 130, then its largest change in the period would be |130-80| = 50.

In [41]:
def part_seven():
    census_df_alt = census_df[census_df['SUMLEV'] == 50]
    census_df_alt['max_pop']=[max(census_df_alt['POPESTIMATE2010'][n],census_df_alt['POPESTIMATE2011'][n],
                              census_df_alt['POPESTIMATE2012'][n],census_df_alt['POPESTIMATE2013'][n],
                              census_df_alt['POPESTIMATE2014'][n],census_df_alt['POPESTIMATE2015'][n])
                          for n in census_df_alt.index]
    census_df_alt['min_pop']=[min(census_df_alt['POPESTIMATE2010'][n],census_df_alt['POPESTIMATE2011'][n],
                              census_df_alt['POPESTIMATE2012'][n],census_df_alt['POPESTIMATE2013'][n],
                              census_df_alt['POPESTIMATE2014'][n],census_df_alt['POPESTIMATE2015'][n])
                          for n in census_df_alt.index]
    census_df_alt['max_diff_pop']=[census_df_alt['max_pop'][n]-census_df_alt['min_pop'][n] for n in census_df_alt.index]
    sorted_max_diff_pop = census_df_alt.sort_values(by='max_diff_pop', ascending=False)
    print("Largest absolute change in population: " + str(sorted_max_diff_pop.iloc[0]['max_diff_pop']))
    return sorted_max_diff_pop.iloc[0]['CTYNAME']
part_seven()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Largest absolute change in population: 429841


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


'Harris County'

### Part 8
In this datafile, the United States is broken up into four regions using the "REGION" column. 

Created a query that finds the counties that belong to regions 1 or 2, whose name starts with 'Washington', and whose POPESTIMATE2015 was greater than their POPESTIMATE 2014.

In [42]:
def part_eight():
    filtered = census_df[census_df.REGION.isin([1,2])& 
              census_df.CTYNAME.str.startswith('Washington')][['STNAME', 'CTYNAME','POPESTIMATE2015','POPESTIMATE2014']]
    filtered['greaterthan2014']=[filtered['POPESTIMATE2015'][n]>census_df['POPESTIMATE2014'][n] for n in filtered.index]
    further_filtered = filtered[filtered.greaterthan2014.eq(True)]
    return further_filtered[['STNAME', 'CTYNAME']]
part_eight()

,STNAME,CTYNAME
896,Iowa,Washington County
1419,Minnesota,Washington County
2345,Pennsylvania,Washington County
2355,Rhode Island,Washington County
3163,Wisconsin,Washington County
